In [59]:
#notebook for engineer features using game results and stats
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import graphviz
from sklearn.tree import export_graphviz
from sklearn import tree
from dtreeviz.trees import dtreeviz

In [2]:
#import Data
PL19_20_fixtures = pd.read_csv('Data/2019-20/Fixtures/PL19_20_fixtures.csv')
PL20_21_fixtures = pd.read_csv('Data/2020-21/Fixtures/PL20_21_fixtures.csv')
PL21_22_fixtures = pd.read_csv('Data/2021-22/Fixtures/PL21_22_fixtures.csv')
PL19_20_fixtures['game_home_ID'] = PL19_20_fixtures['HomeTeam'] + PL19_20_fixtures['Date'].astype(str)
PL19_20_fixtures['game_away_ID'] = PL19_20_fixtures['AwayTeam'] + PL19_20_fixtures['Date'].astype(str)
PL20_21_fixtures['game_home_ID'] = PL20_21_fixtures['HomeTeam'] + PL20_21_fixtures['Date'].astype(str)
PL20_21_fixtures['game_away_ID'] = PL20_21_fixtures['AwayTeam'] + PL20_21_fixtures['Date'].astype(str)
PL21_22_fixtures['game_home_ID'] = PL21_22_fixtures['HomeTeam'] + PL21_22_fixtures['Date'].astype(str)
PL21_22_fixtures['game_away_ID'] = PL21_22_fixtures['AwayTeam'] + PL21_22_fixtures['Date'].astype(str)
PL21_22_fixtures

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Rating_Home,Rating_Away,Rating difference,game_home_ID,game_away_ID
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,2.13,2.05,2.17,1.80,2.09,73,79,-6,Brentford2021-08-13,Arsenal2021-08-13
1,E0,2021-08-14,12:30,Manchester Utd,Leeds United,5,1,H,1,0,...,1.77,2.19,1.93,2.10,1.79,83,76,7,Manchester Utd2021-08-14,Leeds United2021-08-14
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,2.14,1.82,2.19,1.79,2.12,76,75,1,Burnley2021-08-14,Brighton2021-08-14
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,1.81,2.16,1.93,2.06,1.82,83,76,7,Chelsea2021-08-14,Crystal Palace2021-08-14
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,1.88,2.08,1.90,2.03,1.86,79,76,3,Everton2021-08-14,Southampton2021-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,0,...,2.23,1.88,2.25,1.74,2.16,76,83,-7,Crystal Palace2022-05-22,Manchester Utd2022-05-22
376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,0,...,2.03,1.94,2.26,1.87,2.01,80,76,4,Leicester City2022-05-22,Southampton2022-05-22
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,1.83,2.19,1.99,2.07,1.80,84,78,6,Liverpool2022-05-22,Wolves2022-05-22
378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,1,...,1.86,2.09,2.03,2.01,1.87,85,78,7,Manchester City2022-05-22,Aston Villa2022-05-22


In [3]:
#generate gameweeks dependening on the order of the games
def generate_gameweeks(df):
    teams = df.HomeTeam.unique().tolist()
    df['Wk_h'] = 0
    df['Wk_a'] = 0
    for team in teams:
        count = 1
        for index, row in df.iterrows():
            if row['HomeTeam'] == team:
                df.at[index, 'Wk_h'] = count
                count = count + 1
            elif row['AwayTeam'] == team:
                df.at[index, 'Wk_a'] = count
                count = count + 1
    return df
PL19_20_fixtures = generate_gameweeks(PL19_20_fixtures)
PL20_21_fixtures = generate_gameweeks(PL20_21_fixtures)
PL21_22_fixtures = generate_gameweeks(PL21_22_fixtures)

PL21_22_fixtures

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,MaxCAHA,AvgCAHH,AvgCAHA,Rating_Home,Rating_Away,Rating difference,game_home_ID,game_away_ID,Wk_h,Wk_a
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,2.17,1.80,2.09,73,79,-6,Brentford2021-08-13,Arsenal2021-08-13,1,1
1,E0,2021-08-14,12:30,Manchester Utd,Leeds United,5,1,H,1,0,...,1.93,2.10,1.79,83,76,7,Manchester Utd2021-08-14,Leeds United2021-08-14,1,1
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,2.19,1.79,2.12,76,75,1,Burnley2021-08-14,Brighton2021-08-14,1,1
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,1.93,2.06,1.82,83,76,7,Chelsea2021-08-14,Crystal Palace2021-08-14,1,1
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,1.90,2.03,1.86,79,76,3,Everton2021-08-14,Southampton2021-08-14,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,0,...,2.25,1.74,2.16,76,83,-7,Crystal Palace2022-05-22,Manchester Utd2022-05-22,38,38
376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,0,...,2.26,1.87,2.01,80,76,4,Leicester City2022-05-22,Southampton2022-05-22,38,38
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,1.99,2.07,1.80,84,78,6,Liverpool2022-05-22,Wolves2022-05-22,38,38
378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,1,...,2.03,2.01,1.87,85,78,7,Manchester City2022-05-22,Aston Villa2022-05-22,38,38


In [4]:
#functions for feature Engineering

#get points for one game
def get_points(team, wk, df):
    for index, row in df.iterrows():
        if row['Wk_h'] == wk and row['HomeTeam'] == team:
            if row['FTR'] == 'H':
                return 3
            if row['FTR'] == 'D':
                return 1
            else:
                return 0
        if row['Wk_a'] == wk and row['AwayTeam'] == team:
            if row['FTR'] == 'A':
                return 3
            if row['FTR'] == 'D':
                return 1
            else:
                return 0

#get values for statistics (shots, corners, goals)
def get_stats(team, wk, stat, df):
    for index, row in df.iterrows():
        #print(row[['HomeTeam','Wk']])
        if row['Wk_h'] == wk and row['HomeTeam'] == team:
            if stat == 'goals':
                return row['FTHG']
            elif stat == 'shots':
                return row['HST']
            elif stat == 'corners':
                return row['HC']
        elif row['Wk_a'] == wk and row['AwayTeam'] == team:
            if stat == 'goals':
                return row['FTAG']
            elif stat == 'shots':
                return row['AST']
            elif stat == 'corners':
                return row['AC']
            return row[stat]

#sum points over a season
def get_points_season(team, fr, to, df):
    test = []
    out = 0
    df2 = 0
    if fr == to: return 0
    for i in range(fr, to):
        out = out + get_points(team, i, df)
        test.append(out)
        df3 = pd.Series(test)
        df2 = df3.rolling(len(df3)).mean()
        df2 = df2[len(df3) - 1]
    return df2

#sum stats values over a season
def get_stats_season(team, fr, to, category, df):
    test = []
    out = 0
    df2 = 0
    if fr == to: return 0
    for i in range(fr, to):
        out = out + get_stats(team, i, category, df)
        test.append(out)
        df3 = pd.Series(test)
        df2 = df3.rolling(len(df3)).mean()
        df2 = df2[len(df3) - 1]
        #df2 = df.ewm(alpha=0.4).mean()[len(df) - 1]
    return df2

#get ewm over a season
def get_ewm_season(team, fr, to, df):
    test = []
    out = 0
    if fr == to: return 0
    for i in range(fr, to):
        out = out + get_points(team, i, df)
        test.append(out)
        df3 = pd.Series(test)
        df2 = df3.ewm(alpha=0.9).mean()[len(df3) - 1]
    return df2

#append form to df
def append_form(timespan, df):
    formH = []
    formA = []
    for index, row in df.iterrows():
        wk_h = row['Wk_h']
        wk_a = row['Wk_a']
        teamH = row['HomeTeam']
        teamA = row['AwayTeam']
        if int(wk_h) - timespan < 1:
            pointsH = get_points_season(teamH, 1, int(wk_h) - 1, df)
        else:
            pointsH = get_points_season(teamH, int(wk_h) - timespan, int(wk_h) - 1, df=df)
        if int(wk_a) - timespan < 1:
            pointsA = get_points_season(teamA, 1, int(wk_a) - 1, df)
        else:
            pointsA = get_points_season(teamA, int(wk_a) - timespan, int(wk_a) - 1, df=df)
        formH.append(pointsH)
        formA.append(pointsA)
    df['formH'] = formH
    df['formA'] = formA
    return df

#append ewm to df
def append_ewm(timespan, df):
    formH = []
    formA = []
    for index, row in df.iterrows():
        wk_h = row['Wk_h']
        wk_a = row['Wk_a']
        teamH = row['HomeTeam']
        teamA = row['AwayTeam']
        if int(wk_h) - timespan < 1:
            pointsH = get_ewm_season(teamH, 1, int(wk_h), df)
        else:
            pointsH = get_ewm_season(teamH, int(wk_h) - timespan, int(wk_h) - 1, df)
        if int(wk_a) - timespan < 1:
            pointsA = get_ewm_season(teamA, 1, int(wk_a), df)
        else:
            pointsA = get_ewm_season(teamA, int(wk_a) - timespan, int(wk_a) - 1, df)
        formH.append(pointsH)
        formA.append(pointsA)
    df['ewmH'] = formH
    df['ewmA'] = formA
    return df

#append stats to df
def append_stats(timespan, df):
    goals_h = []
    goals_A = []
    shots_H = []
    shots_A = []
    corners_H = []
    corners_A = []

    for index, row in df.iterrows():
        wk_h = row['Wk_h']
        wk_a = row['Wk_a']
        teamH = row['HomeTeam']
        teamA = row['AwayTeam']
        if int(wk_h) - timespan < 1:
            goalsH = get_stats_season(teamH, 1, int(wk_h) - 1, 'goals', df)
            shotsH = get_stats_season(teamH, 1, int(wk_h) - 1, 'shots', df)
            cornersH = get_stats_season(teamH, 1, int(wk_h) - 1, 'corners', df)
        else:
            goalsH = get_stats_season(teamH, int(wk_h) - timespan, int(wk_h) - 1, 'goals', df)
            shotsH = get_stats_season(teamH, int(wk_h) - timespan, int(wk_h) - 1, 'shots', df)
            cornersH = get_stats_season(teamH, int(wk_h) - timespan, int(wk_h) - 1, 'corners', df)

        if int(wk_a) - timespan < 1:
            goalsA = get_stats_season(teamA, 1, int(wk_a) - 1, 'goals', df)
            shotsA = get_stats_season(teamA, 1, int(wk_a) - 1, 'shots', df)
            cornersA = get_stats_season(teamA, 1, int(wk_a) - 1, 'corners', df)
        else:
            goalsA = get_stats_season(teamA, int(wk_a) - timespan, int(wk_a) - 1, 'goals', df)
            shotsA = get_stats_season(teamA, int(wk_a) - timespan, int(wk_a) - 1, 'shots', df)
            cornersA = get_stats_season(teamA, int(wk_a) - timespan, int(wk_a) - 1, 'corners', df)

        goals_h.append(goalsH)
        goals_A.append(goalsA)
        shots_H.append(shotsH)
        shots_A.append(shotsA)
        corners_H.append(cornersH)
        corners_A.append(cornersA)
    df['goalsH'] = goals_h
    df['goalsA'] = goals_A
    df['shotsH'] = shots_H
    df['shotsA'] = shots_A
    df['cornersH'] = corners_H
    df['cornersA'] = corners_A
    return df


#Calculate Elo Rating
def expected_result(loc, aw):
    dr = loc - aw
    we = (1 / (10 ** (-dr / 400) + 1))
    return [np.round(we, 3), 1 - np.round(we, 3)]

def actual_result(loc, aw):
    if loc < aw:
        wa = 1
        wl = 0
    elif loc > aw:
        wa = 0
        wl = 1
    elif loc == aw:
        wa = 0.5
        wl = 0.5
    return [wl, wa]


def calculate_elo(elo_l, elo_v, local_goals, away_goals, k_value):
    k = k_value
    wl, wv = actual_result(local_goals, away_goals)
    wel, wev = expected_result(elo_l, elo_v)

    elo_ln = elo_l + k * (wl - wel)
    elo_vn = elo_v + k * (wv - wev)

    return elo_ln, elo_vn

def calculate_elo_season(df):
    current_elo = {}
    for idx, row in df.iterrows():

        local = row['HomeTeam']
        away = row['AwayTeam']
        local_goals = row['FTHG']
        away_goals = row['FTAG']
        #tournament = row['tournament']

        if local not in current_elo.keys():
            current_elo[local] = 1500

        if away not in current_elo.keys():
            current_elo[away] = 1500

        elo_l = current_elo[local]
        elo_v = current_elo[away]
        elo_ln, elo_vn = calculate_elo(elo_l, elo_v, local_goals, away_goals, 30)

        current_elo[local] = elo_ln
        current_elo[away] = elo_vn

        df.loc[idx, 'Elo_h_after'] = elo_ln
        df.loc[idx, 'Elo_a_after'] = elo_vn
        df.loc[idx, 'Elo_h_before'] = elo_l
        df.loc[idx, 'Elo_a_before'] = elo_v
        return df
calculate_elo_season(PL21_22_fixtures)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Rating_Away,Rating difference,game_home_ID,game_away_ID,Wk_h,Wk_a,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,79,-6,Brentford2021-08-13,Arsenal2021-08-13,1,1,1515.0,1485.0,1500.0,1500.0
1,E0,2021-08-14,12:30,Manchester Utd,Leeds United,5,1,H,1,0,...,76,7,Manchester Utd2021-08-14,Leeds United2021-08-14,1,1,NaN,NaN,NaN,NaN
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,75,1,Burnley2021-08-14,Brighton2021-08-14,1,1,NaN,NaN,NaN,NaN
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,76,7,Chelsea2021-08-14,Crystal Palace2021-08-14,1,1,NaN,NaN,NaN,NaN
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,76,3,Everton2021-08-14,Southampton2021-08-14,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,0,...,83,-7,Crystal Palace2022-05-22,Manchester Utd2022-05-22,38,38,NaN,NaN,NaN,NaN
376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,0,...,76,4,Leicester City2022-05-22,Southampton2022-05-22,38,38,NaN,NaN,NaN,NaN
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,78,6,Liverpool2022-05-22,Wolves2022-05-22,38,38,NaN,NaN,NaN,NaN
378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,1,...,78,7,Manchester City2022-05-22,Aston Villa2022-05-22,38,38,NaN,NaN,NaN,NaN


In [5]:
PL19_20_fixtures = calculate_elo_season(PL19_20_fixtures)
PL21_22_fixtures = calculate_elo_season(PL21_22_fixtures)
PL21_22_fixtures

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,Rating_Away,Rating difference,game_home_ID,game_away_ID,Wk_h,Wk_a,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before
0,E0,2021-08-13,20:00,Brentford,Arsenal,2,0,H,1,0,...,79,-6,Brentford2021-08-13,Arsenal2021-08-13,1,1,1515.0,1485.0,1500.0,1500.0
1,E0,2021-08-14,12:30,Manchester Utd,Leeds United,5,1,H,1,0,...,76,7,Manchester Utd2021-08-14,Leeds United2021-08-14,1,1,NaN,NaN,NaN,NaN
2,E0,2021-08-14,15:00,Burnley,Brighton,1,2,A,1,0,...,75,1,Burnley2021-08-14,Brighton2021-08-14,1,1,NaN,NaN,NaN,NaN
3,E0,2021-08-14,15:00,Chelsea,Crystal Palace,3,0,H,2,0,...,76,7,Chelsea2021-08-14,Crystal Palace2021-08-14,1,1,NaN,NaN,NaN,NaN
4,E0,2021-08-14,15:00,Everton,Southampton,3,1,H,0,1,...,76,3,Everton2021-08-14,Southampton2021-08-14,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,0,...,83,-7,Crystal Palace2022-05-22,Manchester Utd2022-05-22,38,38,NaN,NaN,NaN,NaN
376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,0,...,76,4,Leicester City2022-05-22,Southampton2022-05-22,38,38,NaN,NaN,NaN,NaN
377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,1,...,78,6,Liverpool2022-05-22,Wolves2022-05-22,38,38,NaN,NaN,NaN,NaN
378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,1,...,78,7,Manchester City2022-05-22,Aston Villa2022-05-22,38,38,NaN,NaN,NaN,NaN


In [7]:
#add features to table
PL19_20_fixtures = append_stats(7, PL19_20_fixtures)
PL19_20_fixtures = append_ewm(40, PL19_20_fixtures)
PL19_20_fixtures = append_form(7, PL19_20_fixtures)
PL19_20_fixtures = calculate_elo_season(PL19_20_fixtures)
PL20_21_fixtures = append_stats(7, PL20_21_fixtures)
PL20_21_fixtures = append_ewm(40, PL20_21_fixtures)
PL20_21_fixtures = append_form(7, PL20_21_fixtures)
PL20_21_fixtures = calculate_elo_season(PL20_21_fixtures)
PL21_22_fixtures = append_stats(7, PL21_22_fixtures)
PL21_22_fixtures = append_ewm(40, PL21_22_fixtures)
PL21_22_fixtures = append_form(7, PL21_22_fixtures)
PL21_22_fixtures = calculate_elo_season(PL21_22_fixtures)


KeyboardInterrupt: 

In [52]:
#calculate Differences
def calculate_differences(df):
    df['goalsDifference'] = df['goalsH'] - df['goalsA']
    df['shotsDifference'] = df['shotsH'] - df['shotsA']
    df['cornersDifference'] = df['cornersH'] - df['cornersA']
    df['ewmDifference'] = df['ewmH'] - df['ewmA']
    df['elo_difference'] = df['Elo_h_before'] - df['Elo_a_before']
    df['form_difference'] = df['formH'] - df['formA']
    return df
PL19_20_fixtures = calculate_differences(PL19_20_fixtures)
PL20_21_fixtures = calculate_differences(PL20_21_fixtures)
PL21_22_fixtures = calculate_differences(PL21_22_fixtures)

PL20_21_fixtures

In [2]:
#export features as csv
PL19_20_fixtures.to_csv('Data/2019-20/Fixtures/PL19_20_features.csv')
PL20_21_fixtures.to_csv('Data/2020-21/Fixtures/PL20_21_features.csv')
PL21_22_fixtures.to_csv('Data/2021-22/Fixtures/PL21_22_features.csv')

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before,goalsDifference,shotsDifference,cornersDifference,ewmDifference,elo_difference,form_difference
0,0,E0,2019-08-09,20:00,Liverpool,Norwich City,4,1,H,4,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,1,E0,2019-08-10,12:30,West Ham,Manchester City,0,5,A,0,...,1485.00,1515.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,2,E0,2019-08-10,15:00,Bournemouth,Sheffield Utd,1,1,D,0,...,1500.00,1500.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,3,E0,2019-08-10,15:00,Burnley,Southampton,3,0,H,0,...,1515.00,1485.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,4,E0,2019-08-10,15:00,Crystal Palace,Everton,0,0,D,0,...,1500.00,1500.00,1500.00,1500.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2020-07-26,16:00,Leicester City,Manchester Utd,0,2,A,0,...,1500.87,1598.97,1512.69,1587.15,-7.333333,-10.333333,-8.333333,-0.898797,-74.46,-5.833333
376,376,E0,2020-07-26,16:00,Manchester City,Norwich City,5,0,H,2,...,1637.85,1309.92,1633.74,1314.03,8.333333,13.333333,15.333333,56.666697,319.71,5.500000
377,377,E0,2020-07-26,16:00,Newcastle Utd,Liverpool,1,3,A,1,...,1448.82,1696.44,1454.97,1690.29,-1.166667,-6.166667,-9.166667,-51.798683,-235.32,-2.666667
378,378,E0,2020-07-26,16:00,Southampton,Sheffield Utd,3,1,H,0,...,1522.65,1502.76,1507.17,1518.24,-0.166667,6.000000,-3.333333,-5.307820,-11.07,-0.166667
